# Projeto 2 de Ciência dos Dados

Nome: Ana Carolina Pela Tofetti, Eduardo Lima Pinelli e Rafael Seicali Malcervelli

In [1]:
#Carregando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\rafae\Desktop\cdados_p2\cdados_projeto2


In [163]:
#Lendo o dataset
#Criando um novo dataframe somente com as partidas, mapas e times, excluindo as estatisticas de cada jogador, pois
#existem alguns jogadores que atualmente não estão mais participando do cenário competitivo do jogo.
data = pd.read_csv("matches_info.csv").iloc[:, 0:23]

#Apagando colunas desnecessárias
del data["Unnamed: 0"]
del data["date_match"]
del data["event_name"]

#Esta função faz com que todas as colunas sejam expostas pelo DataFrame.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

#Função que conta a quantidade de aparições dos mapas, retornando um dicionário com cada um deles e suas frequencias
def contagem(lista):
    dicionario_contagem = {}
    for i in lista:
        dicionario_contagem[i] = lista.count(i)
        
    return dicionario_contagem

In [68]:
#Criando uma lista que contenha o nome de todos os times
lista_times1 = list(data_novo.team1)
lista_times2 = list(data_novo.team2)
lista_times = lista_times1+lista_times2

#Criando uma lista que contenha o nome de todos os mapas
lista_mapas = []
lista_frq_mapas = []
lista_crua = list(data_novo.map1_played) + list(data_novo.map2_played) + list(data_novo.map3_played)

for i in lista_crua:
    if i in lista_mapas or i == "NotPlayed":
        pass
    else:
        lista_mapas.append(i)
        
#Criando uma lista para medir a frequencia de jogos nos mapas
for i in lista_crua:
    if i == "NotPlayed":
        pass
    else:
        lista_frq_mapas.append(i)

In [165]:
#Criando um dicionário que mostra a frequencia que cada mapa é jogado
soma_mapas = sum(contagem(lista_frq_mapas).values())
dicionario_frequencias_mapas = {}
for k, v in contagem(lista_frq_mapas).items():
    frq = v*(100) / soma_mapas
    dicionario_frequencias_mapas[k] = frq

#Criando um dicionário que mostra a frequencia dos times em relação as partidas jogadas
soma_times = sum(contagem(lista_times).values())
dicionario_frequencias_times = {}
for k, v in contagem(lista_times).items():
    frq = v*(100) / soma_times
    dicionario_frequencias_times[k] = frq

#Código que cria um dicionário que mostra a porcentagem de vitória dos times a partir da soma de todas as partidas computadas.
lista_mapa2 = []
lista_mapa3 = []

for i in data.team_winner_map2:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa2.append(i)

for i in data.team_winner_map3:
    if i == "NotPlayed":
        pass
    else:
        lista_mapa3.append(i)

vencedores = list(data.team_winner_map1) + lista_mapa2 + lista_mapa3
soma_vencedores = sum(contagem(vencedores).values())
dicionario_frequencias_vencedores = {}
dicionario_vencedores_sorted = {}
dicionario_times_sorted = {}
for k, v in contagem(vencedores).items():
    frq = v*(100) / soma_vencedores
    dicionario_frequencias_vencedores[k] = frq

#Transformando todos as keys em lowercase, para poder ordená-las melhor.
dict_lower_vencedores = {}
for k, v in dicionario_frequencias_vencedores.items():
    key = k.lower()
    dict_lower_vencedores[key] = dicionario_frequencias_vencedores[k]
    
dict_lower_times = {}
for k, v in dicionario_frequencias_times.items():
    key = k.lower()
    dict_lower_times[key] = dicionario_frequencias_times[k]
    
dict_lower_mapas = {}
for k, v in dicionario_frequencias_mapas.items():
    key = k.lower()
    dict_lower_mapas[key] = dicionario_frequencias_mapas[k]
    
#Organizando os dicionários em ordem alfabetica, facilitando sua visualização
dict_lower_vencedores = dict((k, v) for k, v in dict_lower_vencedores.items()) 
for key in sorted(dict_lower_vencedores):
    dicionario_vencedores_sorted[key] = dict_lower_vencedores[key]
    
dict_lower_times = dict((k, v) for k, v in dict_lower_times.items()) 
for key in sorted(dict_lower_times):
    dicionario_times_sorted[key] = dict_lower_times[key]

#Criando um dicionário que consiste em ter como chave os times e como valor, um dicionário de mapas, contendo
#em sua chave os mapas e no valor as porcentagens de vitória que o time teve em cada mapa
dicionario_vencedores_mapas = {}
for k, v in dicionario_times_sorted.items():
    dicionario_vencedores_mapas[k] = dicionario_frequencias_mapas

#Criando um dicionário que consiste em ter como chave os times, e como valor, um dicionário dos times, contendo
#em sua chave os times e no valor as porcentagens de vitória que o time teve com cada um dos times.
for time, v in dicionario_times_sorted.items():
    dicionario_vencedores_times[time] = dicionario_times_sorted
    if time in dicionario_times_sorted:
        dicionario_copia = dicionario_times_sorted.copy()
        dicionario_copia.pop(time)
        dicionario_vencedores_times[time] = dicionario_copia
    


In [166]:
dicionario_vencedores_times

{'astralis': {'avangar': 2.5,
  'big': 3.3333333333333335,
  'cloud9': 4.583333333333333,
  'complexity': 1.6666666666666667,
  'cr4zy': 1.0416666666666667,
  'dreameaters': 0.625,
  'ence': 2.5,
  'envy': 1.0416666666666667,
  'faze': 6.458333333333333,
  'flipsid3': 1.6666666666666667,
  'fnatic': 4.791666666666667,
  'g2': 5.625,
  'gambit': 3.3333333333333335,
  'godsent': 1.0416666666666667,
  'hellraisers': 2.7083333333333335,
  'immortals': 1.6666666666666667,
  'liquid': 5.0,
  'mibr': 3.5416666666666665,
  'mousesports': 4.583333333333333,
  'natus vincere': 6.875,
  'nip': 2.9166666666666665,
  'north': 3.75,
  'nrg': 1.6666666666666667,
  'optic': 0.8333333333333334,
  'penta': 0.8333333333333334,
  'quantum bellator fire': 1.0416666666666667,
  'renegades': 2.5,
  'sk': 3.5416666666666665,
  'space soldiers': 1.0416666666666667,
  'tyloo': 0.8333333333333334,
  'vega squadron': 2.5,
  'virtus.pro': 3.3333333333333335,
  'vitality': 2.0833333333333335,
  'winstrike': 0.625},